# Check CRS extraction

I was having issues with pygrib running into segmentation faults in my GitHub Actions after it was updated to support Numpy v2+. I was only using pygrib to extract coordinate reference system (CRS) data, and I would like to remove it as a dependency.

Here I am comparing the CRS proj parameters extracted by pygrib and what is extracted by Herbie (from the keys read by cfgrib). 

**Since pygrib doesn't work in GitHub actions for me for whatever reason, I should run this notebook before any release to manually test the values between pygrib and Herbie agree.**

In [1]:
from herbie import Herbie
import pygrib
import pandas as pd
from pyproj import CRS
from herbie.crs import get_cf_crs

In [2]:
for model, search in [
    (dict(model="hrrr"), ":TMP:2 m above"),
    (dict(model="hrrrak"), ":TMP:2 m above"),
    (dict(model="gfs"), ":TMP:2 m above"),
    (dict(model="graphcast"), ":TMP:2 m above"),
    (dict(model="cfs", member=1, product="6_hourly", kind="flxf"), ":TMP:2 m above"),
    (dict(model="ifs"), ":2t:"),
    (dict(model="aifs"), ":2t:"),
    (dict(model="gefs", member=1), ":TMP:2 m above"),
    (dict(model="gefs", product="wave", member=1), ":WIND:surf"),
    # FAILING
    # (dict(model="hafsa", product="storm.atm", storm="07s"), ":TMP:2 m above"),
    # FAILING
    # (dict(model="href", products="mean", domain="conus", fxx=1), ":TMP:2 m above"),
    (dict(model="nam"), ":TMP:2 m above"),
    (dict(model="urma"), ":TMP:2 m above"),
    (dict(model="rtma"), ":TMP:2 m above"),
    (dict(model="hrdps", fxx=0, product="continental/2.5km", variable="TMP", level="AGL-2m"), None),

]:
    if model["model"] in ("hafsa", "href"):  # only on nomads
        date = "2025-01-16 06:00"
    elif model["model"] in ("hrdps",): # no historical data
        date = pd.Timestamp("now").floor("6h") - pd.Timedelta("6h")
    else:
        date = "2025-01-01"

    ds = Herbie(date, verbose=False, **model).xarray(
        search, remove_grib=False, _use_pygrib_for_crs=False
    )

    print()
    print(f"Model: {model['model'].upper()}")
    with pygrib.open(str(ds.local_grib)) as grb:
        msg = grb.message(1)
        projparams_dict_pygrib = dict(sorted(msg.projparams.items()))
        print("  pygrib", projparams_dict_pygrib)

    projparams_dict_herbie = dict(
        sorted(get_cf_crs(ds, _return_projparams=True).items())
    )
    print("  Herbie", projparams_dict_herbie)
    print("  equal=", projparams_dict_herbie == projparams_dict_pygrib)


Model: HRRR
  pygrib {'a': 6371229, 'b': 6371229, 'lat_0': 38.5, 'lat_1': 38.5, 'lat_2': 38.5, 'lon_0': 262.5, 'proj': 'lcc'}
  Herbie {'a': 6371229, 'b': 6371229, 'lat_0': 38.5, 'lat_1': 38.5, 'lat_2': 38.5, 'lon_0': 262.5, 'proj': 'lcc'}
  equal= True

Model: HRRRAK
  pygrib {'a': 6371229, 'b': 6371229, 'lat_0': 90.0, 'lat_ts': 60.0, 'lon_0': 225.0, 'proj': 'stere'}
  Herbie {'a': 6371229, 'b': 6371229, 'lat_0': 90, 'lat_ts': 60.0, 'lon_0': 225.0, 'proj': 'stere'}
  equal= True

Model: GFS
  pygrib {'a': 6371229, 'b': 6371229, 'proj': 'longlat'}
  Herbie {'a': 6371229, 'b': 6371229, 'proj': 'longlat'}
  equal= True

Model: GRAPHCAST
  pygrib {'a': 4326.0, 'b': 4326.0, 'proj': 'longlat'}
  Herbie {'a': 4326.0, 'b': 4326.0, 'proj': 'longlat'}
  equal= True

Model: CFS
  pygrib {'a': 6371229, 'b': 6371229, 'proj': 'longlat'}
  Herbie {'a': 6371229, 'b': 6371229, 'proj': 'longlat'}
  equal= True

Model: IFS
  pygrib {'a': 6371229, 'b': 6371229, 'proj': 'longlat'}
  Herbie {'a': 6371229,

## Check the crs accessor

In [2]:
ds = Herbie("2025-1-1").xarray(":TMP:2 m ab")
ds.herbie.crs

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2025-Jan-01 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/home/blaylock/GITHUB/Herbie/herbie/core.py:1117: UserWarning: Will not remove GRIB file because it previously existed.
  warnings.warn("Will not remove GRIB file because it previously existed.")


<Projected CRS: +proj=lcc +ellps=sphere +a=6371229.0 +b=6371229.0  ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Lambert Conic Conformal (2SP)
Datum: unknown
- Ellipsoid: unknown
- Prime Meridian: Greenwich